# Plotting the incidences of BW counties

In [ ]:
import pandas as pd    # Pandas is a library to analyse various data structures
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Here we read the csv-file holding the data.
# Note: the separator is a ; instead of a comma, all NA-values gave to be skipped
df_csv = pd.read_csv('/Users/greiner/Programming/Python/FreiburgSeminar/Infizierte.csv', sep=';',na_filter=False)
keys=df_csv.keys()

In [ ]:
# Let us pack the counties names into a dictionary and number them
# e.g. {'Biberach': 1}
nkreise=np.arange(len(df_csv.to_numpy()[:,0]))
dict_kreise = dict(zip(df_csv.to_numpy()[:,0],nkreise))

In [ ]:
data=df_csv.to_numpy()[:,1:]

In [ ]:
irange=np.arange(data.shape[0])
jrange=np.arange(data.shape[1])
datafloat=np.zeros(data.shape)
for i in irange: 
    for j in jrange:
        if data[i,j] =='': 
            datafloat[i,j]=0.
        else:
            datafloat[i,j]=float(data[i,j])

In [ ]:
%matplotlib notebook
fig, ax = plt.subplots() # let us plot the data
y = np.flip(datafloat[0])
delta = y - np.roll(y,shift=7)
deltabool = delta > 0
ax.plot(delta[deltabool])
ax.set_title('Die 7-Tage Inzidenz')
ax.set(xlabel='Tag')
ax.set(ylabel='Anzahl')

In [ ]:
datafloat[0]